Looking to align output with the output of ottawa_libraries.py.
Specifically:
```
{lib_name:
    {
        Latitude: lat
        Longitude: long
    }
}
```

Data from [here](https://data.calgary.ca/Recreation-and-Culture/Calgary-Public-Library-Locations-and-Hours-Map/niu9-34wk).

#### Pull data

In [ ]:
import urllib.request
import json 

In [ ]:
url = "https://data.calgary.ca/resource/niu9-34wk.json"
calgary_data = urllib.request.urlopen(url).read()
data = json.loads(calgary_data)

In [ ]:
type(data)

In [ ]:
len(data)

In [ ]:
type(data[0])

In [ ]:
data[0].keys()

In [ ]:
import pprint

In [ ]:
pprint.pprint(data[0])

In [ ]:
pprint.pprint(data)

In [ ]:
calgary_libraries = {}
for lib in data:
    location = {}
    location['Latitude'] = lib['location']['latitude']
    location['Longitude'] = lib['location']['longitude']
    calgary_libraries[lib['library']] = location

In [ ]:
pprint.pprint(calgary_library)

#### See if we can map easily

In [ ]:
%matplotlib inline
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import ottawa_libraries as ol

In [ ]:
# open and plot the data. looks like Toronto
census_file = 'data/census_tracts.shp'
canada_census = gpd.read_file(census_file)

In [ ]:
calgary_census = canada_census[(canada_census.CMANAME=="Calgary")]

In [ ]:
libraries = pd.DataFrame(calgary_library)
# pivot the dataframe
libraries = libraries.T.reset_index()
libraries = libraries.rename(columns={'index':'Name'})

In [ ]:
libraries['Latitude'] = pd.to_numeric(libraries['Latitude'])
libraries['Longitude'] = pd.to_numeric(libraries['Longitude'])

In [ ]:
libraries['geometry'] = libraries.apply(lambda x:
                                                shapely.geometry.point.Point(x['Longitude'], x['Latitude']),
                                                axis=1)
libraries_gpd = gpd.GeoDataFrame(libraries)

In [ ]:
m = calgary_census.explore()
libraries_gpd.explore(m=m, color='red')

#### Add in density

In [ ]:
pop_data = pd.read_csv('data/98-401-X2016043_eng_CSV/98-401-X2016043_English_CSV_data.csv')

In [ ]:
calgary_pop_data = pop_data[pop_data['GEO_CODE (POR)'].isin(calgary_census.CTUID)]

In [ ]:
calgary_pop_data = calgary_pop_data.loc[calgary_pop_data["DIM: Profile of Census Tracts (2247)"]=='Population, 2016']
calgary_pop_data['population'] = calgary_pop_data['Dim: Sex (3): Member ID: [1]: Total - Sex']
calgary_pop_data['population'] = pd.to_numeric(calgary_pop_data['population'])
calgary_pop_data['population'] = pd.to_numeric(calgary_pop_data['population'])

calgary_census['CTUID'] = pd.to_numeric(calgary_census['CTUID'])
calgary_census = calgary_census.merge(right=calgary_pop_data, how='left',
                              left_on='CTUID', right_on='GEO_CODE (POR)')

In [ ]:
calgary_census['density'] = calgary_census['population']/calgary_census.area

In [ ]:
calgary_census_display = calgary_census[['geometry', 'population', 'density']]
m = calgary_census_display.explore(column='density')
libraries_gpd.explore(m=m, color='red')